In [1]:
# Draft of a Resnet to input LR saturation time images and output HR T1 maps 

from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import add
from keras.utils import plot_model
from keras.layers import Input, Dense, Add
from keras import activations, initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization
import keras
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

Using TensorFlow backend.


In [2]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channelsx=9,
                 n_channelsy=9, shuffle=True, phase='training'):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channelsx = n_channelsx
        self.n_channelsy = n_channelsy
        self.shuffle = shuffle
        self.phase = phase
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channelsx))
        y = np.empty((self.batch_size, *self.dim, self.n_channelsy))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('C://Users//310304075//Desktop//DataResnet//' + self.phase + '//LR_' + ID + '.npy')

            # Store class
            y[i,] = np.load('C://Users//310304075//Desktop//DataResnet//' + self.phase + '//HR_' + ID + '.npy')
    
        print (np.shape(X))
        print (self.phase)
        print (np.shape(y))
        return X, y    
    
def convolutional_block(X, f, filters, stage, block, s = 1):
    """
    Implementation of the convolutional block, resnet
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1 = filters 
    F2 = filters
    F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

def relu_bn(inputs):
    
    """
    Batch normalizationa and RELU
    """
    bn = BatchNormalization(axis=-1, center=True, scale=False)(inputs)
    bn = Activation('relu')(bn)
    return bn


def residual_module(layer_in):
    merge_input = layer_in
    merge_input = Conv2D(2, (1,1), padding='same', activation='relu', input_shape=(patch_size, patch_size, numberTS), kernel_initializer='he_normal')(merge_input)
    merge_input = relu_bn(merge_input)
    
    Res1 = convolutional_block (merge_input, 3, 4, 1, "Res1", 1)
    Res2 = convolutional_block (Res1, 3, 8, 2, "Res2", 1)
    Res3 = convolutional_block (Res2, 3, 16, 3, "Res3", 1)
    Res4 = convolutional_block (Res3, 3, 32, 4, "Res4", 1)
    Res5 = convolutional_block (Res4, 3, 8, 5, "Res5", 1)
    Res6 = convolutional_block (Res5, 3, 2, 6, "Res6", 1)
            
    layer_out = Conv2D(3, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(Res6)
    
    return layer_out
    
patch_size = 336
numberTS = 9
batch_size = 4

# define model input
visible = Input(batch_shape=(batch_size, patch_size, patch_size, numberTS))
# add vgg module
layer = residual_module(visible)
# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
#model.summary()

model.compile(SGD(lr=0.01, momentum=0.95), loss = 'mse')

# save the model and weights
model_name = 'ResNetT1'
outdir="C://Users//310304075//Desktop//ResnetOutputs//"
weights_filepath = os.path.join(outdir, "weights//resnetT1_{epoch:03d}_{val_loss:.2f}.hdf5")
model_filepath = os.path.join(outdir, 'ResnetT1.json')

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json) 

# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(log_dir = os.path.join(outdir, 'logs//ResNetT1//'))
callbacks_list = [checkpoint, tensorboard]


# train the model
#train_steps = train_gen.n//train_gen.batch_size
#val_steps = val_gen.n//val_gen.batch_size

ID={}
ID["train"] = ['1', '2', '3', '4','5','6','7','8','9','10']
ID["validation"] = ['1', '2']

# Parameters
paramstr = {'dim': (336,336),
          'batch_size': 4,
          'n_channelsx': 9,
          'n_channelsy': 3,
          'shuffle': True,
          'phase':"training"}

paramsval = {'dim': (336,336),
          'batch_size': 2,
          'n_channelsx': 9,
          'n_channelsy': 3,
          'shuffle': False,
          'phase':"validation"}

# Datasets
partition = ID

# Generators
training_generator = DataGenerator(partition['train'], **paramstr)
validation_generator = DataGenerator(partition['validation'], **paramsval)

history = model.fit_generator(generator=training_generator, steps_per_epoch=2, 
                    validation_data=validation_generator,
                    epochs=3,
                    callbacks=callbacks_list
                    )

for key in history.history:
    print(key)


Epoch 1/3
(2, 336, 336, 9)
validation
(2, 336, 336, 3)
(4, 336, 336, 9)
training
(4, 336, 336, 3)
(4, 336, 336, 9)
training
(4, 336, 336, 3)
1/2 [==============>...............] - ETA: 31s - loss: 864031187402752.0000(4, 336, 336, 9)
training
(4, 336, 336, 3)
(4, 336, 336, 9)
training
(4, 336, 336, 3)
(2, 336, 336, 9)
validation
(2, 336, 336, 3)
2/2 [==============================] - 45s 22s/step - loss: 864031187402752.0000 - val_loss: 8670047006359552.0000

Epoch 00001: val_loss improved from inf to 8670047006359552.00000, saving model to C://Users//310304075//Desktop//ResnetOutputs//weights//resnetT1_001_8670047006359552.00.hdf5
Epoch 2/3
1/2 [==============>...............] - ETA: 11s - loss: 864031187402752.0000(4, 336, 336, 9)
training
(4, 336, 336, 3)
(4, 336, 336, 9)
training
(4, 336, 336, 3)
(2, 336, 336, 9)
validation
(2, 336, 336, 3)
2/2 [==============================] - 26s 13s/step - loss: 864031187402752.0000 - val_loss: 432015593701376.0000

Epoch 00002: val_loss improv